# Breakout

In [15]:
import gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import SubprocVecEnv, VecFrameStack, DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import os

## Env

In [63]:
ENV_NAME = 'Breakout-v0'

#### Testing

In [27]:
env = gym.make(ENV_NAME)

episodes = 5
for ep in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    
    print(f"Ep #{ep} - Score: {score}")
env.close()

C:\Users\mehul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


Ep #1 - Score: 3.0
Ep #2 - Score: 3.0
Ep #3 - Score: 1.0
Ep #4 - Score: 2.0
Ep #5 - Score: 2.0


### Vectorized Stack of Envs

In [79]:
# stack env to train on all at the same time
env = make_atari_env(ENV_NAME, n_envs=4, seed=0)
env = VecFrameStack(env, n_stack=4)

## Model

### Initializing

In [75]:
LOG_PATH = os.path.join('training', 'logs', ENV_NAME)

In [61]:
# new
model = A2C('CnnPolicy', env, verbose=1, tensorboard_log=LOG_PATH)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [80]:
# load
MODEL_PATH = os.path.join('training', 'saved_models', ENV_NAME, 'v1')
model = A2C.load(MODEL_PATH, env=env)

Wrapping the env in a VecTransposeImage.


### Training

In [76]:
model.learn(total_timesteps=10000)

Logging to training\logs\breakout\A2C_3
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 480      |
|    ep_rew_mean        | 5.89     |
| time/                 |          |
|    fps                | 123      |
|    iterations         | 100      |
|    time_elapsed       | 16       |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -0.79    |
|    explained_variance | 0.917    |
|    learning_rate      | 0.0007   |
|    n_updates          | 5149     |
|    policy_loss        | 0.1      |
|    value_loss         | 0.109    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 474      |
|    ep_rew_mean        | 5.8      |
| time/                 |          |
|    fps                | 124      |
|    iterations         | 200      |
|    time_elapsed       | 32       |
|    total_timesteps    | 4000     

In [77]:
# save model
SAVE_PATH = os.path.join('training', 'saved_models', ENV_NAME, 'v1')
model.save(SAVE_PATH)
del model

### Evaluation

In [81]:
env = make_atari_env(ENV_NAME, n_envs=1, seed=0)
env = VecFrameStack(env, n_stack=4)

In [84]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(5.1, 1.57797338380595)

In [85]:
env.close()